<a href="https://colab.research.google.com/github/ytaylor/connect-to-api-elsevier/blob/master/search_information_citations.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
!pip install --upgrade -q gspread beautifulsoup4

In [0]:
from google.colab import drive, auth
drive.mount('/content/gdrive')
GDRIVE_BASE_DIR = '/content/gdrive/My Drive/MY_WORK/03_Search_information_citations/'
print(GDRIVE_BASE_DIR)

In [0]:
auth.authenticate_user()
import gspread 

from oauth2client.client import GoogleCredentials

gc = gspread.authorize(GoogleCredentials.get_application_default())

In [0]:
sh = gc.open('data_dhvis')
wh = sh.worksheet("citations")
wh.update_acell('O1', 'doi_elsevier')

### Retrieve info for a document API ELsevier

In [0]:
import requests
import json
    
## Load configuration
con_file = open(GDRIVE_BASE_DIR+"config.json")
config = json.load(con_file)
con_file.close()


## Find information empty fields citations
citations_wh = sh.worksheet('citations')
citations_rows = citations_wh.get_all_values()
citations_df = pd.DataFrame.from_records(citations_rows[1:], columns=citations_rows[0])

### Example for test results

In [0]:
string_title= 'Exploring the Betrothed Lovers' 
title = 'TITLE(\"' + string_title + '\")'
params = {'query': title}
response = requests.get("https://api.elsevier.com/content/abstract/scopus_id/84905855329",
                    headers={'Accept':'application/json', 
                             'X-ELS-APIKey': config['apikey']})
if response.status_code ==200:
  #results= response.json()
  results = json.loads(response.text.encode('utf-8'))
  print(response.json())
      
  #If exists publication name
  try:
    publicationName= results['search-results']['entry'][0]['dc:description']     
    print(publicationName)
  except KeyError:
    print("Oops! No era válido. Intente nuevamente...")

{'abstracts-retrieval-response': {'affiliation': [{'affiliation-city': 'Trento', 'affilname': 'Cross Library Services srl', 'affiliation-country': 'Italy'}, {'affiliation-city': 'Torino', 'affilname': 'Celi S.R.L.', 'affiliation-country': 'Italy'}, {'affiliation-city': 'Novara', 'affilname': 'Universita degli Studi del Piemonte Orientale Amedeo Avogadro', 'affiliation-country': 'Italy'}, {'affiliation-city': 'Torino', 'affilname': 'Universita degli Studi di Torino', 'affiliation-country': 'Italy'}], 'coredata': {'srctype': 'p', 'eid': '2-s2.0-84905855329', 'prism:coverDate': '2013-01-01', 'prism:aggregationType': 'Conference Proceeding', 'prism:url': 'https://api.elsevier.com/content/abstract/scopus_id/84905855329', 'subtypeDescription': 'Conference Paper', 'dc:creator': {'author': [{'ce:given-name': 'Andrea', 'preferred-name': {'ce:given-name': 'Andrea', 'ce:initials': 'A.', 'ce:surname': 'Bolioli', 'ce:indexed-name': 'Bolioli A.'}, '@seq': '1', 'ce:initials': 'A.', '@_fa': 'true', 'a

###Items with publication_title empty

In [0]:
#Items with publication_title empty
for index, row in citations_df.iterrows():
    if row['publication_title']=="": 
      string_title= row['title']; 
      title = 'TITLE(\"' + string_title + '\")'
      params = {'query': title}
      response = requests.get("https://api.elsevier.com/content/search/scopus", params=params,
                      headers={'Accept':'application/json', 
                               'X-ELS-APIKey': config['apikey']})
      print(response)
      if response.status_code ==200:
        #results= response.json()
        results = json.loads(response.text.encode('utf-8'))

        #If exists publication name
        try:
          publicationName= results['search-results']['entry'][0]['prism:publicationName']
          year = results['search-results']['entry'][0]['prism:coverDisplayDate']
          #Actualizar la informacion en la fila del wh
          wh.update_acell('O{}'.format(index+2), publicationName)
          print(publicationName)

        except KeyError:
          print("Oops! No era válido. Intente nuevamente...")
          pass


###Items with doi empty

In [0]:
 #If not exists doi
for index, row in citations_df.iterrows(): 
  if row['DOI']=="" :
    string_title= row['title']; 
    title = 'TITLE(\"' + string_title + '\")'
    params = {'query': title}
    response = requests.get("https://api.elsevier.com/content/search/scopus", params=params,
                        headers={'Accept':'application/json', 
                                 'X-ELS-APIKey': config['apikey']})
    print(string_title)
    if response.status_code ==200:
      results = json.loads(response.text.encode('utf-8'))
      try:
        doi= results['search-results']['entry'][0]['prism:doi']
        #Actualizar la informacion en la fila del wh
        wh.update_acell('O{}'.format(index+2), doi)
        print(doi)
      except KeyError:
        print("Oops! No era válido. Intente nuevamente...")
        pass

###Items with abstract empty

In [0]:
#If not exist abstratc
for index, row in citations_df.iterrows(): 
  if row['abstract']=="" :
    string_title= row['title']; 
    title = 'TITLE(\"' + string_title + '\")'
    params = {'query': title}
    response = requests.get("https://api.elsevier.com/content/search/scopus", params=params,
                        headers={'Accept':'application/json', 
                                 'X-ELS-APIKey': config['apikey']})
    results = json.loads(response.text.encode('utf-8'))
    print(title)
    if response.status_code ==200:
      results = json.loads(response.text.encode('utf-8'))
      try:
        abstract= results['search-results']['entry'][0]['dc:description']
        print (abstract)
        #Actualizar la informacion en la fila del wh
        wh.update_acell('O{}'.format(index+2), abstract)
        print(doi)
      except KeyError:
        print("Oops! No era válido. Intente nuevamente...")
        pass

### Scrapping Web

In [0]:
from bs4 import BeautifulSoup

payload = {'inUserName': 'ytaylor@usal.es', 'inUserPass': 'taylor*89'}


with requests.Session() as s:
    p = s.post('https://www.scopus.com/home.uri', data=payload)
    # print the html returned or something more intelligent to see if it's a successful login page.

# An authorised request.
    r = s.get('https://www.scopus.com/record/display.uri?eid=2-s2.0-84905855329&origin=resultslist&sort=plf-f&src=s&st1=Exploring+the+Betrothed+Lovers&st2=&sid=e1b0b384135d12796766e7f967ef4418&sot=b&sdt=b&sl=37&s=TITLE%28Exploring+the+Betrothed+Lovers%29&relpos=0&citeCnt=2&searchTerm=')
    print(r.text)



<script type="text/javascript" src="https://acw.elsevier.com/SSOCore/jawr_loader.js"></script>
<script type="text/javascript" src="/gzip_N806072365/bundles/masterjquery.js" ></script>
<script type="text/javascript">
JAWR.loader.script('/bundles/jcars.js');
JAWR.loader.style('/bundles/jcars.css', 'all');
</script>
<script language="javascript" type="text/javascript">
window.$ = window.jQuery;
</script>
<script type="text/javascript" src="/gzip_N1606155294/bundles/remoteAccMsg.js" ></script>
<script>
var feMetricUrl = new Object();
feMetricUrl = 'https://els-eum.elsst.com';
</script>
<link rel="stylesheet" type="text/css" media="all" href="/gzip_1190653315/bundles/ScopusMasterLayout.css" />
<script type="text/javascript" src="/gzip_354756789/bundles/AllTop.js" ></script>
<script src="https://b2btagmgr.azalead.com/tag?az=aWRTdWJzY3JpYmVyPTIyMjYmaWRTaXRlPTIzMTYmb3JpZ2luPXd3dy5lbHNldmllci5jb20mY3VzdG9tPW5vbmU=" type="application/javascript" async> </script> 
<script type="text/javascript">